In [1]:
'''
To execute this file an updated version of chromedriver is required,
it can be downloaded here https://chromedriver.chromium.org/downloads
after downloading place it in the same directory as this file

run 
!pip install pandas
!pip install selenium
!pip install beautifulsoup4
'''
#importing libraries
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import csv
from datetime import date
import pandas as pd

#dictionary for dates formatting
months = {'Jan.': '01', 
          'Feb.': '02',
          'Mar.': '03',
          'Apr.': '04',
          'May': '05',
          'Jun.': '06',
          'Jul.': '07',
          'Aug.': '08',
          'Sep.': '09',
          'Oct.': '10',
          'Nov.': '11',
          'Dec.': '12',    
}
#dates formatting 
def make_date(d):
    parts = d.split(' ')[1:] #get rid of the weekday
    
    for key, el in months.items():
        if parts[0] == key: parts[0] = el
    if len(parts) == 2:
        parts.append('2023')
        made_date = str(parts[2]) + '-' + str(parts[0]) + '-' + str(parts[1])
    else:
        made_date = str(parts[2]) + '-' + str(parts[0]) + '-' + str(parts[1][:-1])
    return made_date
    
def parcing(url):
    matched = []
    news = []
    dates = []
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        driver = webdriver.Chrome(options=options)
        driver.get(url) 
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        for h3 in soup.find_all('h3')[1:]:
            news.append(h3.text) #getting all news from a page
        '''    
        unfortunately, span classes are swithced often, therefore 
        "class":"lM_Gf lM_jd" may not be appropriate
        if you want to run this code please contact with me and I'll 
        share the instruction on how to find the appropriate class/the appropriate class itself
        '''
        for span in soup.find_all('span', {"class":"lM_Gf lM_jd"})[1:]:
            dates.append(make_date(span.text)) #getting all dates corresponding to the news
        for i in range(len(dates)):
            matched.append([dates[i], news[i]]) #matching dates-news
    except Exception as ex:
        print(ex)
    finally:
        return matched
        driver.close()
        driver.quit()   

def main():
    all_news = []
    make_link = "https://seekingalpha.com/symbol/ATVI/news"
    all_news += parcing(url=make_link) #parsing first page
    for page_num in range(2, 31):
        time.sleep(20) #pause inbetween opening page is required due to the restrictions of the cite
        make_link = "https://seekingalpha.com/symbol/ATVI/news" + "?page=" + str(page_num) #parsing every page 2-31
        all_news += parcing(url=make_link)
    df = pd.DataFrame(all_news)
    df.to_csv(path_or_buf="data.csv", index=False)
    
if __name__ == '__main__':
    main()
